# <center>spherical_demo.ipynb
<h1> <center>WESmith 06/29/22

In [1]:
import operator_utils as ws
import sympy as sy

# <center> 3D spherical coordinates </center>

In [2]:
aa = ws.coord_xforms()
dd = ws.Operators(aa['spherical_3d'])

In [3]:
title = 'The transform used in {} coordinates:'.format(dd.name)
ws.latex_block(dd.transform, title=title, center=True, lhs=dd.coords, h_title=1, h_eq=2)

# <center> The transform used in spherical_3d coordinates: </center>
## $$ \left( x, \  y, \  z\right) = \left( r \sin{\left(\theta \right)} \cos{\left(\phi \right)}, \  r \sin{\left(\phi \right)} \sin{\left(\theta \right)}, \  r \cos{\left(\theta \right)}\right) $$

In [4]:
ws.latex_block(dd.Jacobian, title='The Jacobian in {} coordinates:'.format(dd.name), lhs='J')

# <center> The Jacobian in spherical_3d coordinates: </center>
#### $$ \mathtt{\text{J}} = \left[\begin{matrix}\sin{\left(\theta \right)} \cos{\left(\phi \right)} & r \cos{\left(\phi \right)} \cos{\left(\theta \right)} & - r \sin{\left(\phi \right)} \sin{\left(\theta \right)}\\\sin{\left(\phi \right)} \sin{\left(\theta \right)} & r \sin{\left(\phi \right)} \cos{\left(\theta \right)} & r \sin{\left(\theta \right)} \cos{\left(\phi \right)}\\\cos{\left(\theta \right)} & - r \sin{\left(\theta \right)} & 0\end{matrix}\right] $$

In [5]:
# simp() reduces expressions element-by-element
ws.latex_block(ws.simp(dd.Jacobian_inv), title='The inverse Jacobian in {} coordinates:'.format(dd.name), lhs='inv(J)', h_eq=3)

# <center> The inverse Jacobian in spherical_3d coordinates: </center>
### $$ \mathtt{\text{inv(J)}} = \left[\begin{matrix}\sin{\left(\theta \right)} \cos{\left(\phi \right)} & \sin{\left(\phi \right)} \sin{\left(\theta \right)} & \cos{\left(\theta \right)}\\\frac{\cos{\left(\phi \right)} \cos{\left(\theta \right)}}{r} & \frac{\sin{\left(\phi \right)} \cos{\left(\theta \right)}}{r} & - \frac{\sin{\left(\theta \right)}}{r}\\- \frac{\sin{\left(\phi \right)}}{r \sin{\left(\theta \right)}} & \frac{\cos{\left(\phi \right)}}{r \sin{\left(\theta \right)}} & 0\end{matrix}\right] $$

In [6]:
title = 'The primed basis in {} coordinates:'.format(dd.name)
ws.latex_block(dd.primed_basis, title=title, lhs=dd.Zp, center=True, h_title=1, h_eq=3)

# <center> The primed basis in spherical_3d coordinates: </center>
### $$ \left[\begin{matrix}\hat{\mathbf{e_r}}\\\hat{\mathbf{e_\theta}}\\\hat{\mathbf{e_\phi}}\end{matrix}\right] = \left[\begin{matrix}\hat{\mathbf{e_x}} \sin{\left(\theta \right)} \cos{\left(\phi \right)} + \hat{\mathbf{e_y}} \sin{\left(\phi \right)} \sin{\left(\theta \right)} + \hat{\mathbf{e_z}} \cos{\left(\theta \right)}\\\hat{\mathbf{e_x}} \cos{\left(\phi \right)} \cos{\left(\theta \right)} + \hat{\mathbf{e_y}} \sin{\left(\phi \right)} \cos{\left(\theta \right)} - \hat{\mathbf{e_z}} \sin{\left(\theta \right)}\\- \hat{\mathbf{e_x}} \sin{\left(\phi \right)} + \hat{\mathbf{e_y}} \cos{\left(\phi \right)}\end{matrix}\right] $$

In [7]:
ws.latex_block(dd.metric, title='The metric in {} coordinates:'.format(dd.name), lhs='g', h_title=1, h_eq=3)

# <center> The metric in spherical_3d coordinates: </center>
### $$ \mathtt{\text{g}} = \left[\begin{matrix}1 & 0 & 0\\0 & r^{2} & 0\\0 & 0 & r^{2} \sin^{2}{\left(\theta \right)}\end{matrix}\right] $$

In [8]:
ws.latex_block(dd.metric_inv, title='The inverse metric in {} coordinates:'.format(dd.name), lhs='inv(g)', h_title=1, h_eq=3)

# <center> The inverse metric in spherical_3d coordinates: </center>
### $$ \mathtt{\text{inv(g)}} = \left[\begin{matrix}1 & 0 & 0\\0 & \frac{1}{r^{2}} & 0\\0 & 0 & \frac{1}{r^{2} \sin^{2}{\left(\theta \right)}}\end{matrix}\right] $$

In [9]:
fname = dd._scalar_field('F')
LHS = dd.gradient_symbolic(fname)
title='The gradient of a general scalar field in {} coordinates:'.format(dd.name)
ws.latex_block(dd.gradient(fname), title=title, lhs=LHS, h_title=1, h_eq=3)

# <center> The gradient of a general scalar field in spherical_3d coordinates: </center>
### $$ \nabla{\left(F{\left(r,\theta,\phi \right)} \right)} = \left[\begin{matrix}\frac{\partial}{\partial r} F{\left(r,\theta,\phi \right)}\\\frac{\frac{\partial}{\partial \theta} F{\left(r,\theta,\phi \right)}}{r}\\\frac{\frac{\partial}{\partial \phi} F{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}}\end{matrix}\right] $$

In [10]:
title = 'The gradient represented in the {} basis:'.format(dd.name)
ws.latex_block(dd.gradient(fname, vector=False), title=title, lhs=LHS, h_title=1, h_eq=3)

# <center> The gradient represented in the spherical_3d basis: </center>
### $$ \nabla{\left(F{\left(r,\theta,\phi \right)} \right)} = \left[\begin{matrix}\frac{\hat{\mathbf{e_\phi}} \frac{\partial}{\partial \phi} F{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}} + \frac{\hat{\mathbf{e_\theta}} \frac{\partial}{\partial \theta} F{\left(r,\theta,\phi \right)}}{r} + \hat{\mathbf{e_r}} \frac{\partial}{\partial r} F{\left(r,\theta,\phi \right)}\end{matrix}\right] $$

In [11]:
vname = dd._vector_field()
LHS   = dd.divergence_symbolic(vname)
title = 'The divergence of a general vector field in {} coordinates:'.format(dd.name)
ws.latex_block(dd.divergence(vname), title=title, lhs=LHS, h_title=1, h_eq=4)

# <center> The divergence of a general vector field in spherical_3d coordinates: </center>
#### $$ \nabla\bullet{\left(\left[\begin{matrix}\operatorname{V_{r}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}\end{matrix}\right] \right)} = \frac{\partial}{\partial r} \operatorname{V_{r}}{\left(r,\theta,\phi \right)} + \frac{\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)} \cos{\left(\theta \right)}}{r \sin{\left(\theta \right)}} + \frac{2 \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r} + \frac{\frac{\partial}{\partial \theta} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r} + \frac{\frac{\partial}{\partial \phi} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}} $$

In [12]:
LHS   = dd.curl_symbolic(vname)
title = 'The curl of a general vector field in {} coordinates:'.format(dd.name)
ws.latex_block(dd.curl(vname), title=title, lhs=LHS, h_title=1, h_eq=4)

# <center> The curl of a general vector field in spherical_3d coordinates: </center>
#### $$ \nabla\times{\left(\left[\begin{matrix}\operatorname{V_{r}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}\end{matrix}\right] \right)} = \left[\begin{matrix}\frac{\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)} \cos{\left(\theta \right)}}{r \sin{\left(\theta \right)}} + \frac{\frac{\partial}{\partial \theta} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r} - \frac{\frac{\partial}{\partial \phi} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}}\\- \frac{\partial}{\partial r} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)} - \frac{\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r} + \frac{\frac{\partial}{\partial \phi} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}}\\\frac{\partial}{\partial r} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)} + \frac{\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r} - \frac{\frac{\partial}{\partial \theta} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r}\end{matrix}\right] $$

In [13]:
LHS   = dd.curl_symbolic(vname)
title = 'The curl of a general vector field represented in the {} basis:'.format(dd.name)
ws.latex_block(dd.curl(vname, vector=False), title=title, lhs=LHS, h_title=1, h_eq=4)

# <center> The curl of a general vector field represented in the spherical_3d basis: </center>
#### $$ \nabla\times{\left(\left[\begin{matrix}\operatorname{V_{r}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}\end{matrix}\right] \right)} = \left[\begin{matrix}\hat{\mathbf{e_\phi}} \left(\frac{\partial}{\partial r} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)} + \frac{\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r} - \frac{\frac{\partial}{\partial \theta} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r}\right) + \hat{\mathbf{e_\theta}} \left(- \frac{\partial}{\partial r} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)} - \frac{\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r} + \frac{\frac{\partial}{\partial \phi} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}}\right) + \hat{\mathbf{e_r}} \left(\frac{\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)} \cos{\left(\theta \right)}}{r \sin{\left(\theta \right)}} + \frac{\frac{\partial}{\partial \theta} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r} - \frac{\frac{\partial}{\partial \phi} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}}\right)\end{matrix}\right] $$

In [14]:
LHS   = dd.Laplacian_symbolic(fname)
title = 'The Laplacian of a general scalar field represented in {} coordinates:'.format(dd.name)
ws.latex_block(dd.Laplacian_scalar(fname), title=title, lhs=LHS, h_title=1, h_eq=3)

# <center> The Laplacian of a general scalar field represented in spherical_3d coordinates: </center>
### $$ \nabla^{2}{\left(F{\left(r,\theta,\phi \right)} \right)} = \frac{\partial^{2}}{\partial r^{2}} F{\left(r,\theta,\phi \right)} + \frac{2 \frac{\partial}{\partial r} F{\left(r,\theta,\phi \right)}}{r} + \frac{\frac{\partial^{2}}{\partial \theta^{2}} F{\left(r,\theta,\phi \right)}}{r^{2}} + \frac{\frac{\partial}{\partial \theta} F{\left(r,\theta,\phi \right)}}{r^{2} \tan{\left(\theta \right)}} + \frac{\frac{\partial^{2}}{\partial \phi^{2}} F{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} $$

In [15]:
r, theta, phi = dd.coords_p

In [16]:
# separable function
f, g, h, F = sy.symbols('f g h F', cls=sy.Function)
f = f(r)
g = g(theta)
h = h(phi)
F = f * g * h

In [17]:
LHS   = dd.Laplacian_symbolic(F)
title = 'The Laplacian of a separable scalar field in {} coordinates'.format(dd.name)
ws.latex_block(dd.Laplacian_scalar(F), title=title, lhs=dd.Laplacian_symbolic(F), h_title=1, h_eq=3)

# <center> The Laplacian of a separable scalar field in spherical_3d coordinates </center>
### $$ \nabla^{2}{\left(f{\left(r \right)} g{\left(\theta \right)} h{\left(\phi \right)} \right)} = g{\left(\theta \right)} h{\left(\phi \right)} \frac{d^{2}}{d r^{2}} f{\left(r \right)} + \frac{2 g{\left(\theta \right)} h{\left(\phi \right)} \frac{d}{d r} f{\left(r \right)}}{r} + \frac{f{\left(r \right)} g{\left(\theta \right)} \frac{d^{2}}{d \phi^{2}} h{\left(\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{f{\left(r \right)} h{\left(\phi \right)} \frac{d^{2}}{d \theta^{2}} g{\left(\theta \right)}}{r^{2}} + \frac{f{\left(r \right)} h{\left(\phi \right)} \frac{d}{d \theta} g{\left(\theta \right)}}{r^{2} \tan{\left(\theta \right)}} $$

In [18]:
title = 'The Laplacian of a vector field in {} coordinates (compressed form):'.format(dd.name)
ws.latex_block(dd.Laplacian_vector_reduced(vname), title=title, lhs=dd.Laplacian_symbolic(vname), h_eq=3)

# <center> The Laplacian of a vector field in spherical_3d coordinates (compressed form): </center>
### $$ \nabla^{2}{\left(\left[\begin{matrix}\operatorname{V_{r}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}\end{matrix}\right] \right)} = \left[\begin{matrix}\nabla^{2}{\left(\operatorname{V_{r}}{\left(r,\theta,\phi \right)} \right)} - \frac{2 \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \tan{\left(\theta \right)}} - \frac{2 \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2}} - \frac{2 \frac{\partial}{\partial \theta} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2}} - \frac{2 \frac{\partial}{\partial \phi} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin{\left(\theta \right)}}\\\nabla^{2}{\left(\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)} \right)} - \frac{\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{2 \frac{\partial}{\partial \theta} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2}} - \frac{2 \cos{\left(\theta \right)} \frac{\partial}{\partial \phi} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}}\\\nabla^{2}{\left(\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)} \right)} - \frac{\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{2 \frac{\partial}{\partial \phi} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2} \sin{\left(\theta \right)}} + \frac{2 \cos{\left(\theta \right)} \frac{\partial}{\partial \phi} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}}\end{matrix}\right] $$

In [19]:
title = 'The Laplacian of a vector field in {} coordinates (full form):'.format(dd.name)
ws.latex_block(dd.Laplacian_vector_full(vname), title=title, lhs=dd.Laplacian_symbolic(vname), h_eq=3)

# <center> The Laplacian of a vector field in spherical_3d coordinates (full form): </center>
### $$ \nabla^{2}{\left(\left[\begin{matrix}\operatorname{V_{r}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}\\\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}\end{matrix}\right] \right)} = \left[\begin{matrix}\frac{\partial^{2}}{\partial r^{2}} \operatorname{V_{r}}{\left(r,\theta,\phi \right)} + \frac{2 \frac{\partial}{\partial r} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r} - \frac{2 \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \tan{\left(\theta \right)}} - \frac{2 \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2}} - \frac{2 \frac{\partial}{\partial \theta} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2}} + \frac{\frac{\partial^{2}}{\partial \theta^{2}} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2}} + \frac{\frac{\partial}{\partial \theta} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2} \tan{\left(\theta \right)}} - \frac{2 \frac{\partial}{\partial \phi} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin{\left(\theta \right)}} + \frac{\frac{\partial^{2}}{\partial \phi^{2}} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}}\\\frac{\partial^{2}}{\partial r^{2}} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)} + \frac{2 \frac{\partial}{\partial r} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r} - \frac{\operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{\frac{\partial^{2}}{\partial \theta^{2}} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2}} + \frac{2 \frac{\partial}{\partial \theta} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2}} + \frac{\frac{\partial}{\partial \theta} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \tan{\left(\theta \right)}} - \frac{2 \cos{\left(\theta \right)} \frac{\partial}{\partial \phi} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{\frac{\partial^{2}}{\partial \phi^{2}} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}}\\\frac{\partial^{2}}{\partial r^{2}} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)} + \frac{2 \frac{\partial}{\partial r} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r} - \frac{\operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{\frac{\partial^{2}}{\partial \theta^{2}} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2}} + \frac{\frac{\partial}{\partial \theta} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \tan{\left(\theta \right)}} + \frac{2 \frac{\partial}{\partial \phi} \operatorname{V_{r}}{\left(r,\theta,\phi \right)}}{r^{2} \sin{\left(\theta \right)}} + \frac{2 \cos{\left(\theta \right)} \frac{\partial}{\partial \phi} \operatorname{V_{\theta}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}} + \frac{\frac{\partial^{2}}{\partial \phi^{2}} \operatorname{V_{\phi}}{\left(r,\theta,\phi \right)}}{r^{2} \sin^{2}{\left(\theta \right)}}\end{matrix}\right] $$

In [20]:
v1_name = 'A'
v2_name = 'B'
V1 = dd._vector_field(v1_name)
V2 = dd._vector_field(v2_name)
txt = '({}\\bullet\\nabla){}'.format(v1_name, v2_name)
LHS = sy.symbols(txt) # TODO: need to put this into a 'symbolic_material_derivative' method
title = 'The material derivative of two vector fields in {} coordinates:'.format(dd.name)
ws.latex_block(dd.material_derivative(A=V1, B=V2, vector=True), title=title, lhs=LHS, h_eq=3)

# <center> The material derivative of two vector fields in spherical_3d coordinates: </center>
### $$ (A\bullet\nabla)B = \left[\begin{matrix}\operatorname{A_{r}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial r} \operatorname{B_{r}}{\left(r,\theta,\phi \right)} - \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \operatorname{B_{\phi}}{\left(r,\theta,\phi \right)}}{r} + \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial \phi} \operatorname{B_{r}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}} - \frac{\operatorname{A_{\theta}}{\left(r,\theta,\phi \right)} \operatorname{B_{\theta}}{\left(r,\theta,\phi \right)}}{r} + \frac{\operatorname{A_{\theta}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial \theta} \operatorname{B_{r}}{\left(r,\theta,\phi \right)}}{r}\\\operatorname{A_{r}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial r} \operatorname{B_{\theta}}{\left(r,\theta,\phi \right)} - \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \operatorname{B_{\phi}}{\left(r,\theta,\phi \right)} \cos{\left(\theta \right)}}{r \sin{\left(\theta \right)}} + \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial \phi} \operatorname{B_{\theta}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}} + \frac{\operatorname{A_{\theta}}{\left(r,\theta,\phi \right)} \operatorname{B_{r}}{\left(r,\theta,\phi \right)}}{r} + \frac{\operatorname{A_{\theta}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial \theta} \operatorname{B_{\theta}}{\left(r,\theta,\phi \right)}}{r}\\\operatorname{A_{r}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial r} \operatorname{B_{\phi}}{\left(r,\theta,\phi \right)} + \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \operatorname{B_{\theta}}{\left(r,\theta,\phi \right)} \cos{\left(\theta \right)}}{r \sin{\left(\theta \right)}} + \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \operatorname{B_{r}}{\left(r,\theta,\phi \right)}}{r} + \frac{\operatorname{A_{\phi}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial \phi} \operatorname{B_{\phi}}{\left(r,\theta,\phi \right)}}{r \sin{\left(\theta \right)}} + \frac{\operatorname{A_{\theta}}{\left(r,\theta,\phi \right)} \frac{\partial}{\partial \theta} \operatorname{B_{\phi}}{\left(r,\theta,\phi \right)}}{r}\end{matrix}\right] $$

# MISC EXAMPLES

In [21]:
dd.gradient(F)

Matrix([
[                 g(theta)*h(phi)*Derivative(f(r), r)],
[           f(r)*h(phi)*Derivative(g(theta), theta)/r],
[f(r)*g(theta)*Derivative(h(phi), phi)/(r*sin(theta))]])

In [22]:
# Laplacian of f(r) only
dd.Laplacian_scalar(f)

Derivative(f(r), (r, 2)) + 2*Derivative(f(r), r)/r

In [23]:
dd.Laplacian_scalar(1/f)

-Derivative(f(r), (r, 2))/f(r)**2 + 2*Derivative(f(r), r)**2/f(r)**3 - 2*Derivative(f(r), r)/(r*f(r)**2)

In [24]:
dd.divergence(dd.gradient(f*g*h)) # also the Laplacian: products of functions

g(theta)*h(phi)*Derivative(f(r), (r, 2)) + 2*g(theta)*h(phi)*Derivative(f(r), r)/r + f(r)*g(theta)*Derivative(h(phi), (phi, 2))/(r**2*sin(theta)**2) + f(r)*h(phi)*Derivative(g(theta), (theta, 2))/r**2 + f(r)*h(phi)*cos(theta)*Derivative(g(theta), theta)/(r**2*sin(theta))

In [25]:
dd.Laplacian_vector_full(dd.gradient(h))

Matrix([
[           -2*Derivative(h(phi), (phi, 2))/(r**3*sin(theta)**2)],
[-2*cos(theta)*Derivative(h(phi), (phi, 2))/(r**3*sin(theta)**3)],
[              Derivative(h(phi), (phi, 3))/(r**3*sin(theta)**3)]])

In [26]:
dd.curl(dd.curl())

Matrix([
[Derivative(A_\theta(r, theta, phi), r, theta)/r + cos(theta)*Derivative(A_\theta(r, theta, phi), r)/(r*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi, r)/(r*sin(theta)) + A_\theta(r, theta, phi)*cos(theta)/(r**2*sin(theta)) + Derivative(A_\theta(r, theta, phi), theta)/r**2 - Derivative(A_r(r, theta, phi), (theta, 2))/r**2 - cos(theta)*Derivative(A_r(r, theta, phi), theta)/(r**2*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)) - Derivative(A_r(r, theta, phi), (phi, 2))/(r**2*sin(theta)**2)],
[                                                                                                                                                                                  -Derivative(A_\theta(r, theta, phi), (r, 2)) - 2*Derivative(A_\theta(r, theta, phi), r)/r + Derivative(A_r(r, theta, phi), r, theta)/r + Derivative(A_\phi(r, theta, phi), phi, theta)/(r**2*sin(theta)) + cos(theta)*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2) - Derivative(A

In [27]:
dd.gradient(dd.divergence())

Matrix([
[   Derivative(A_r(r, theta, phi), (r, 2)) + 2*Derivative(A_r(r, theta, phi), r)/r + Derivative(A_\theta(r, theta, phi), r, theta)/r + cos(theta)*Derivative(A_\theta(r, theta, phi), r)/(r*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi, r)/(r*sin(theta)) - A_\theta(r, theta, phi)*cos(theta)/(r**2*sin(theta)) - 2*A_r(r, theta, phi)/r**2 - Derivative(A_\theta(r, theta, phi), theta)/r**2 - Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta))],
[Derivative(A_r(r, theta, phi), r, theta)/r - A_\theta(r, theta, phi)/r**2 - A_\theta(r, theta, phi)*cos(theta)**2/(r**2*sin(theta)**2) + Derivative(A_\theta(r, theta, phi), (theta, 2))/r**2 + 2*Derivative(A_r(r, theta, phi), theta)/r**2 + cos(theta)*Derivative(A_\theta(r, theta, phi), theta)/(r**2*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi, theta)/(r**2*sin(theta)) - cos(theta)*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2)],
[                                                                                    

In [28]:
def Arfken_example_2_4_2(ops):
    # ops is an Operators object defined with standard spherical coords r, theta, phi
    # this was successfully validated against Arfken result p. 83 6/18/22
    r, theta, phi = ops.coords_p
    Aphi = sy.symbols('A_phi', cls=sy.Function, real=True)
    Aphi = Aphi(r, theta)
    A    = sy.Matrix([0, 0, Aphi])
    v1   = ops.curl(ops.curl(A))
    tmp  = []
    for j, k in zip(v1, A):
        tmp.append(j + ops.Laplacian_scalar(k) - ops.Laplacian_symbolic(k))
    reduced = sy.trigsimp(ops.Zp.T * sy.Matrix(tmp))
    full    = sy.trigsimp(ops.Zp.T * v1)
    return full, reduced

In [29]:
full, reduced = Arfken_example_2_4_2(dd)
full

Matrix([[\hat{\mathbf{e_\phi}}*(-Derivative(A_phi(r, theta), (r, 2)) - 2*Derivative(A_phi(r, theta), r)/r + A_phi(r, theta)/(r**2*sin(theta)**2) - Derivative(A_phi(r, theta), (theta, 2))/r**2 - Derivative(A_phi(r, theta), theta)/(r**2*tan(theta)))]])

In [30]:
reduced

Matrix([[\hat{\mathbf{e_\phi}}*(-\nabla^{2}(A_phi(r, theta)) + A_phi(r, theta)/(r**2*sin(theta)**2)) - \hat{\mathbf{e_\theta}}*\nabla^{2}(0) - \hat{\mathbf{e_r}}*\nabla^{2}(0)]])

# Basic checks

In [31]:
dd.Laplacian_scalar(1/r)  # should be 0

0

In [32]:
dd.gradient(1/r, vector=False)  # should be -1/r**2 in the r_hat direction

Matrix([[-\hat{\mathbf{e_r}}/r**2]])

In [33]:
dd.curl(dd.gradient())  # should be 0 vector

Matrix([
[0],
[0],
[0]])

In [34]:
#(xx.T * xx).args[2][0]
dd.curl(dd.gradient())._eval_is_zero_matrix()

True

In [35]:
dd.divergence(dd.curl())  # should be 0

0